#Import Libaray

In [190]:
from functools import reduce
import numpy as np  # noqa
import pandas as pd  # noqa
from pandas import DataFrame

# --------------------------------
# Add your lib to import here
import talib.abstract as ta
from datetime import datetime
from freqtrade.exchange import timeframe_to_prev_date
from technical.util import resample_to_interval,resampled_merge
import ccxt
import math


# Fetch Crypto

In [191]:
ex = ccxt.binance()

timeframe = "1m"
def ohlcv_info():
    x = ex.fetch_ohlcv('1000BONK/USDT:USDT', timeframe, limit=50)
    df = pd.DataFrame(x, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    print(f'*** {datetime.now()}  ***\n {df.tail(10)}\n')
    return df

dataframe = ohlcv_info()

*** 2024-02-23 11:12:31.099188  ***
                   date      open      high       low     close     volume
40 2024-02-23 11:03:00  0.011800  0.011849  0.011794  0.011836  4412918.0
41 2024-02-23 11:04:00  0.011839  0.011849  0.011821  0.011836  4790115.0
42 2024-02-23 11:05:00  0.011834  0.011834  0.011795  0.011802  2491067.0
43 2024-02-23 11:06:00  0.011800  0.011804  0.011783  0.011791  3472823.0
44 2024-02-23 11:07:00  0.011790  0.011819  0.011776  0.011808  3935059.0
45 2024-02-23 11:08:00  0.011808  0.011824  0.011794  0.011800  3114337.0
46 2024-02-23 11:09:00  0.011800  0.011808  0.011786  0.011802  2121718.0
47 2024-02-23 11:10:00  0.011802  0.011814  0.011794  0.011813  3168725.0
48 2024-02-23 11:11:00  0.011814  0.011814  0.011804  0.011810  1923828.0
49 2024-02-23 11:12:00  0.011811  0.011817  0.011810  0.011813  2218533.0



Calculate Deviation

In [207]:
def calcDev(length:int,dataframe:DataFrame,index:int):
    logSource = dataframe['close'].apply(lambda x: math.log(x))
    period_1 = length -1
    sumX = 0.0
    sumXX = 0.0
    sumYX = 0.0
    sumY = 0.0
    for i in range(1,length+1):
        lSrc = logSource[index+1-i]
        sumX += i
        sumXX += i * i
        sumYX += i * lSrc
        sumY += lSrc

    slope = np.nan_to_num((length * sumYX - sumX * sumY) / (length * sumXX - sumX * sumX))
    average = sumY / length
    intercept = average - (slope * sumX / length) + slope
    sumDev = 0.0
    sumDxx = 0.0
    sumDyy = 0.0
    sumDyx = 0.0
    regres = intercept + slope * period_1 * 0.5
    sumSlp = intercept

    for i in range(1,period_1+1):
        lSrc = logSource[index+1-i]
        dxt = lSrc - average
        dyt  = sumSlp - regres
        lSrc   -= sumSlp
        sumSlp += slope
        sumDxx +=  dxt * dxt
        sumDyy +=  dyt * dyt
        sumDyx +=  dxt * dyt
        sumDev += lSrc * lSrc

    unStdDev = math.sqrt(sumDev / period_1)
    divisor  = sumDxx * sumDyy
    pearsonR = np.nan_to_num(sumDyx / math.sqrt(divisor))

    return unStdDev,pearsonR,slope,intercept

Calculate Adaptive Trend In Last Candle

In [208]:
periods = np.array([0,5,6])

def calculate_trend_direction(x,dataframe):
    # if(x.name == len(dataframe)-1) | (x.name == len(dataframe)-2):
    if (x.name >= periods[2]):
        devMultiplier = 2.0
        # Calculate Deviation,PersionR,Slope,Intercept
        stdDev01, pearsonR01, slope01, intercept01 = calcDev(periods[1],dataframe,x.name)
        stdDev02, pearsonR02, slope02, intercept02 = calcDev(periods[2],dataframe,x.name)
        # stdDev03, pearsonR03, slope03, intercept03 = calcDev(periods[3],dataframe)
        # stdDev04, pearsonR04, slope04, intercept04 = calcDev(periods[4],dataframe)
        # stdDev05, pearsonR05, slope05, intercept05 = calcDev(periods[5],dataframe)
        # stdDev06, pearsonR06, slope06, intercept06 = calcDev(periods[6],dataframe)
        # stdDev07, pearsonR07, slope07, intercept07 = calcDev(periods[7],dataframe)
        # stdDev08, pearsonR08, slope08, intercept08 = calcDev(periods[8],dataframe)
        # stdDev09, pearsonR09, slope09, intercept09 = calcDev(periods[9],dataframe)
        # stdDev10, pearsonR10, slope10, intercept10 = calcDev(periods[10],dataframe)
        # stdDev11, pearsonR11, slope11, intercept11 = calcDev(periods[11],dataframe)
        # stdDev12, pearsonR12, slope12, intercept12 = calcDev(periods[12],dataframe)
        # stdDev13, pearsonR13, slope13, intercept13 = calcDev(periods[13],dataframe)
        # stdDev14, pearsonR14, slope14, intercept14 = calcDev(periods[14],dataframe)
        # stdDev15, pearsonR15, slope15, intercept15 = calcDev(periods[15],dataframe)
        # stdDev16, pearsonR16, slope16, intercept16 = calcDev(periods[16],dataframe)
        # stdDev17, pearsonR17, slope17, intercept17 = calcDev(periods[17],dataframe)
        # stdDev18, pearsonR18, slope18, intercept18 = calcDev(periods[18],dataframe)
        # stdDev19, pearsonR19, slope19, intercept19 = calcDev(periods[19],dataframe)

        # Find the highest Pearson's R
        highestPearsonR = max(pearsonR01, pearsonR02,
                            #    pearsonR03, pearsonR04, pearsonR05, pearsonR06, pearsonR07, pearsonR08, pearsonR09, pearsonR10, pearsonR11, pearsonR12, pearsonR13, pearsonR14, pearsonR15, pearsonR16, pearsonR17, pearsonR18, pearsonR19
                            )

        # Determine selected length, slope, intercept, and deviations
        detectedPeriod  = 0
        detectedSlope   = 0
        detectedIntrcpt = 0
        detectedStdDev  = 0

        if highestPearsonR == pearsonR01:
            detectedPeriod = periods[1]
            detectedSlope = slope01
            detectedIntrcpt = intercept01
            detectedStdDev = stdDev01
        elif highestPearsonR == pearsonR02:
            detectedPeriod = periods[2] 
            detectedSlope = slope02
            detectedIntrcpt = intercept02
            detectedStdDev = stdDev02
        # elif highestPearsonR == pearsonR03:
        #     detectedPeriod = periods[3]  
        #     detectedSlope = slope03
        #     detectedIntrcpt = intercept03
        #     detectedStdDev = stdDev03
        # elif highestPearsonR == pearsonR04:
        #     detectedPeriod = periods[4]  
        #     detectedSlope = slope04
        #     detectedIntrcpt = intercept04
        #     detectedStdDev = stdDev04
        # elif highestPearsonR == pearsonR05:
        #     detectedPeriod = periods[5]  
        #     detectedSlope = slope05
        #     detectedIntrcpt = intercept05
        #     detectedStdDev = stdDev05
        # elif highestPearsonR == pearsonR06:
        #     detectedPeriod = periods[6]       
        #     detectedSlope = slope06
        #     detectedIntrcpt = intercept06
        #     detectedStdDev = stdDev06
        # elif highestPearsonR == pearsonR07:
        #     detectedPeriod = periods[7]      
        #     detectedSlope = slope07
        #     detectedIntrcpt = intercept07
        #     detectedStdDev = stdDev07
        # elif highestPearsonR == pearsonR08:
        #     detectedPeriod = periods[8]       
        #     detectedSlope = slope08
        #     detectedIntrcpt = intercept08
        #     detectedStdDev = stdDev08
        # elif highestPearsonR == pearsonR09:
        #     detectedPeriod = periods[9]       
        #     detectedSlope = slope09
        #     detectedIntrcpt = intercept09
        #     detectedStdDev = stdDev09
        # elif highestPearsonR == pearsonR10:
        #     detectedPeriod = periods[10]
        #     detectedSlope = slope10
        #     detectedIntrcpt = intercept10
        #     detectedStdDev = stdDev10
        # elif highestPearsonR == pearsonR11:
        #     detectedPeriod = periods[11]
        #     detectedSlope = slope11
        #     detectedIntrcpt = intercept11
        #     detectedStdDev = stdDev11
        # elif highestPearsonR == pearsonR12:
        #     detectedPeriod = periods[12]
        #     detectedSlope = slope12
        #     detectedIntrcpt = intercept12
        #     detectedStdDev = stdDev12
        # elif highestPearsonR == pearsonR13:
        #     detectedPeriod = periods[13]
        #     detectedSlope = slope13
        #     detectedIntrcpt = intercept13
        #     detectedStdDev = stdDev13
        # elif highestPearsonR == pearsonR14:
        #     detectedPeriod = periods[14]
        #     detectedSlope = slope14
        #     detectedIntrcpt = intercept14
        #     detectedStdDev = stdDev14
        # elif highestPearsonR == pearsonR15:
        #     detectedPeriod = periods[15]
        #     detectedSlope = slope15
        #     detectedIntrcpt = intercept15
        #     detectedStdDev = stdDev15
        # elif highestPearsonR == pearsonR16:
        #     detectedPeriod = periods[16]
        #     detectedSlope = slope16
        #     detectedIntrcpt = intercept16
        #     detectedStdDev = stdDev16
        # elif highestPearsonR == pearsonR17:
        #     detectedPeriod = periods[17]
        #     detectedSlope = slope17
        #     detectedIntrcpt = intercept17
        #     detectedStdDev = stdDev17
        # elif highestPearsonR == pearsonR18:
        #     detectedPeriod = periods[18]
        #     detectedSlope = slope18
        #     detectedIntrcpt = intercept18
        #     detectedStdDev = stdDev18
        # elif highestPearsonR == pearsonR19:
        #     detectedPeriod = periods[19]
        #     detectedSlope = slope19
        #     detectedIntrcpt = intercept19
        #     detectedStdDev = stdDev19
        else:
            # Default case
            raise Exception("Cannot Find Highest PearsonR") 
        
        # Calculate start and end price based on detected slope and intercept
        startPrice = math.exp(detectedIntrcpt + detectedSlope * (detectedPeriod - 1))
        endPrice = math.exp(detectedIntrcpt)
        startAtBar = len(dataframe) - detectedPeriod + 1

        # Calculate Upper Upper Price and Upper End price
        upperStartPrice = startPrice * math.exp(devMultiplier * detectedStdDev)
        upperEndPrice   =   endPrice * math.exp(devMultiplier * detectedStdDev)

        # Calculate Lower Price and Lower End Price
        lowerStartPrice = startPrice / math.exp(devMultiplier * detectedStdDev)
        lowerEndPrice =   endPrice / math.exp(devMultiplier * detectedStdDev)

        # Calculate If Uptrend or Downtrend and how strength is this trend
        # Also Know how many this trend exist with period
        # ====== Strategies ======
        # If EndPrice > StartPrice Uptrend
        # If EndPrice < StartPrice Downtrend
        trend_direction = endPrice - startPrice
        return (trend_direction,detectedPeriod,highestPearsonR,startPrice)
    else:
        return (0,0,0)

Trend Finder Main Function

In [210]:
a = datetime.now()
dataframe['trend_direction'] = dataframe.apply((lambda x: calculate_trend_direction(x,dataframe)),axis=1)
dataframe['trend_direction_temp'] = dataframe['trend_direction'].apply(lambda x: x[0])
dataframe.loc[
    (
        (dataframe['trend_direction_temp'].shift(1) < 0)
        &
        (dataframe['trend_direction_temp'] > 0)
    ),
    'trend_direction_real'] = 1

dataframe.loc[
    (
        (dataframe['trend_direction_temp'].shift(1) > 0)
        &
        (dataframe['trend_direction_temp'] < 0)
    ),
    'trend_direction_real'] = -1

b = datetime.now()
print(dataframe['trend_direction_real'].tail(40))
# print(dataframe['trend_direction'][len(dataframe)-40])
print((b - a).microseconds * 0.001)

10    NaN
11    NaN
12   -1.0
13    1.0
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23   -1.0
24   -1.0
25    1.0
26    NaN
27   -1.0
28    NaN
29    NaN
30    NaN
31    1.0
32    NaN
33    NaN
34    NaN
35    NaN
36    NaN
37   -1.0
38    NaN
39   -1.0
40    1.0
41    NaN
42    NaN
43   -1.0
44    NaN
45    NaN
46    1.0
47    NaN
48    NaN
49    NaN
Name: trend_direction_real, dtype: float64
26.426000000000002


In [39]:
from decimal import Decimal

from pyparsing import Optional

from freqtrade.persistence.trade_model import Trade
from freqtrade.util.ft_precise import FtPrecise

def calculate_deviation(num1, num2):
    diff = abs(Decimal(str(num1)) - Decimal(str(num2)))
    diff_str = str(diff).lstrip('0').replace('.', '')
    deviation_result = int(diff_str)
    return deviation_result

def add_deviation(number,isShort,deviation):
    num_str = str(number)
    integer_part, decimal_part = num_str.split('.')
    last_three_digits = decimal_part[-3:]
    last_three_digits_int = int(last_three_digits)
    new_last_three_digits = 0
    if isShort == False:
        new_last_three_digits = str(last_three_digits_int + deviation).zfill(3)
    if isShort == True:
        new_last_three_digits = str(last_three_digits_int - deviation).zfill(3)
    new_last_three_digits = new_last_three_digits.replace("-","")
    new_decimal_part = decimal_part[:-3] + new_last_three_digits
    new_number_str = integer_part + '.' + new_decimal_part
    new_number = float(new_number_str)
    return new_number
    
def calculate_profit_ratio(open_rate,exit_rate,is_short,leverage,stake_ammount):
    quantity = (stake_ammount*leverage)/open_rate
    initial_margin = quantity * open_rate * (1/leverage)
    pnl = 0
    roi = 0
    if(is_short == False):
        pnl = (exit_rate - open_rate) * quantity
    else:
        pnl = (open_rate - exit_rate) * quantity

    roi = pnl / initial_margin
    return round(roi * 100,2)

# Example usage:
num1 = 0.022304
num2 = 0.022314

profit_percent = calculate_profit_ratio(num1,num2,False,20,10)
deviation_result = calculate_deviation(num1, num2)
result  = add_deviation(num2,True,deviation_result)
print("Deviation result:", profit_percent)

Deviation result: 0.9
